# Juan Mario Ochoa Navarro
# Zurisadai Velazquez Manzanero
# Carlos Agustin Manzo Rosas

In [1]:
%pylab inline
from gplearn.genetic import SymbolicRegressor
from gplearn.genetic import SymbolicTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import graphviz
#sklearn.datasets.load_boston(*, return_X_y=False)
import pandas as pd
from sklearn.linear_model import Ridge



Populating the interactive namespace from numpy and matplotlib


In [2]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston = pd.read_csv(r'housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [3]:
data = pd.DataFrame(boston.iloc[:,0:-2])
target = pd.DataFrame(boston.iloc[:,-1])
target.columns = ['MEDV']

In [4]:

function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']


gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0)
gp.fit(data, target)

C:\Users\juanm\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.13         0.319134        6         0.731698         0.337402     23.16s
   1     6.92         0.520272        9         0.760776         0.839335     21.64s
   2     5.31         0.616178       11         0.797053         0.816954     19.66s
   3     7.22         0.616916       11          0.80675         0.705538     18.94s
   4     9.62         0.613597       14         0.835007         0.494405     18.78s
   5    12.60          0.62647       19         0.838189         0.595118     18.85s
   6    15.68         0.650284       29         0.843105         0.527507     18.14s
   7    17.07         0.658093       19         0.854679         0.455565     16.38s
   8    18.81         0.672838       18         0.847042         0.327535  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

In [5]:
gp_features = gp.transform(boston.iloc[:,0:-2])
new_boston = np.hstack((boston.iloc[:,0:-2], gp_features))

### Ridge Regression

In [47]:
est = Ridge()
est.fit(new_boston[:300, :], boston.iloc[:300,-1])
print(est.score(new_boston[:300, :], boston.iloc[:300,-1]))

0.8962996047968077


R^2 score is 89.63%

Now we compare the socre without a transformation

In [48]:
Y = boston.iloc[:,-1]
X = boston.iloc[:,:-2]

In [53]:
import statsmodels.api as sm
X = sm.add_constant(X)
mod = sm.OLS(Y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.684
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     89.01
Date:                Wed, 16 Feb 2022   Prob (F-statistic):          4.90e-115
Time:                        16:24:38   Log-Likelihood:                -1548.6
No. Observations:                 506   AIC:                             3123.
Df Residuals:                     493   BIC:                             3178.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.6526      5.368      3.848      0.0

C:\Users\juanm\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [56]:
#print("Parameters: ", res.params)
print("R2: ", res.rsquared)

R2:  0.6842042799773889


R^2 is way lower than with the transformation, Therefore the transformation for this case was a good idea to implement.